# Export

Parse notebook and extract exportable code cells into corresponding TS modules (directives shamelessly copied from `nbdev`)

In [1]:
//| export

import { z } from "npm:zod@^3.23.8";
import path from "node:path";


In [2]:
//| export

const cellSchema = z.object({
  cell_type: z.enum(["code", "markdown"]),
  source: z.array(z.string()),
});

const nbSchema = z.object({
  cells: z.array(cellSchema)
});

type Cell = z.infer<typeof cellSchema>;
type Nb = z.infer<typeof nbSchema>;

const is_directive = (ln: string) : boolean => ln.replaceAll(" ", "").startsWith("//|"); 
const is_cell_exportable = (cell: Cell) : boolean => cell.cell_type === "code" && cell.source.length > 0 && is_directive(cell.source[0])
  && cell.source[0].includes("export");

export const exportNb = async (nb_path: string): Promise<void> => {
  const module_name = path.basename(nb_path).replace(".ipynb", ".ts");
  const nb = nbSchema.parse(JSON.parse(await Deno.readTextFile(nb_path)));
  const export_cells = nb.cells.filter(cell => is_cell_exportable(cell));
  await Deno.writeTextFile(
    module_name,
    export_cells.reduce((acc, cell) => acc + cell.source.filter(s => !is_directive(s)).join(""), `//source ${nb_path}\n\n`)
  );
}

In [3]:
await exportNb("./export.ipynb");

In [4]:
import { assertEquals } from "jsr:@std/assert";

Deno.test("is_directive", () => {
  assertEquals(is_directive("//| export"), true);
  assertEquals(is_directive("const c = 1;"), false);
  assertEquals(is_directive("// | export"), true);
  assertEquals(is_directive("// |    export"), true);
});

Deno.test("is_cell_exportable", () => {
  assertEquals(is_cell_exportable({cell_type: "code", source: ["//| export\n"]}), true);
  assertEquals(is_cell_exportable({cell_type: "code", source: ["const c = 1;"]}), false);
  assertEquals(is_cell_exportable({cell_type: "code", source: ["//|export\n"]}), true);
  assertEquals(is_cell_exportable({cell_type: "markdown", source: ["# showing //| export\n"]}), false);
  assertEquals(is_cell_exportable({cell_type: "code", source: ["//|export"]}), true);
});


is_directive ... ok (0ms)
is_cell_exportable ... ok (0ms)

ok | 2 passed | 0 failed (0ms)
